## Projet de carte interactive des resultats Législatives et Européenes

### Imports

In [1]:
import geopandas as gpd
import pandas as pd 
import folium
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup
from folium import plugins
import sys
sys.path.append('scripts')  # Add the scripts directory to the system path
from import_function import import_txt_file, process_and_merge_scores, process_and_merge_scores_eur

### Legislatives 1<sup>er</sup> tour 2022

#### Import csv https://www.data.gouv.fr/fr/datasets/elections-legislatives-des-12-et-19-juin-2022-resultats-definitifs-du-premier-tour/

In [2]:
file_path = "Source/2022/resultats-par-niveau-burvot-t1-france-entiere.txt"
Scores_Leg_t1_2022 = import_txt_file(file_path, 'Legislatives_2022', 1)

#### Transposing columns

In [3]:
Scores_Leg_t1_2022_2 = process_and_merge_scores(Scores_Leg_t1_2022)

### Européennes 2024

#### Import csv https://www.data.gouv.fr/fr/datasets/resultats-des-elections-europeennes-du-9-juin-2024/

In [ ]:
file_path = "Source/2024/resultats-definitifs-par-bureau-de-vote.csv"
Scores_Eur_2024 = import_txt_file(file_path, 'Européennes_2024',1, encoding='utf-8')

#### Restriction Paris

In [ ]:
Scores_Eur_2024_75 = Scores_Eur_2024[(Scores_Eur_2024['Code département'] == '75')]

#### Transposing columns

In [ ]:
Scores_Eur_2024_75_2 = process_and_merge_scores_eur(Scores_Eur_2024_75)

In [ ]:
#Scores_Eur_2024_752 = Scores_Eur_2024_75_2[Scores_Eur_2024_75_2['Code BV'].str.startswith(('16', '17'))].assign(
Scores_Eur_2024_752 = Scores_Eur_2024_75_2.assign(
    **{'Code BV': lambda df: df['Code BV'].apply(
        lambda x: f"{int(str(x)[:2])}-{int(str(x)[2:])}" if len(str(x)) == 4 else f"{int(str(x)[:2])}-{int(str(x)[2:])}"
    )}
).rename(columns={"Code BV": "id_bv"})

### Zoom Paris

In [ ]:
Leg_t1_2022_75 = Scores_Leg_t1_2022_2[
    (Scores_Leg_t1_2022_2['Code du département'] == '75')
    #& (Scores_Leg_t1_2022['Code de la circonscription'].isin([3, 4])
].dropna(axis=1, how='all').assign(
    **{'Code du b.vote': lambda df: df['Code du b.vote'].apply(
        lambda x: f"{int(str(x)[:2])}-{int(str(x)[2:])}" if len(str(x)) == 4 else f"{int(str(x)[:2])}-{int(str(x)[2:])}"
    )}
).rename(columns={"Code du b.vote": "id_bv"})

In [ ]:
BDV_Map = gpd.read_file("Source/secteurs-des-bureaux-de-vote.geojson")
BDV = pd.read_csv("Source/ListeBDVParisv5.csv")
BDV4 = BDV.drop('Unnamed: 0', axis=1)
BDV4 = BDV4.rename(columns={"Num": "num_bv", "Arrondissement": "arrondissement"})
BDV4b = BDV4.drop_duplicates(["arrondissement","num_bv","Circonscription"])
BDV4b = BDV4b.map(str)
BDV_Map_4 = pd.merge(BDV_Map,BDV4b,how='left',left_on=['arrondissement',"num_bv"],right_on=['arrondissement',"num_bv"])

In [ ]:
BDV_Map_4[BDV_Map_4['arrondissement']=='4']

In [ ]:
BDV_Map_75=BDV_Map_4.dissolve(by='id_bv')[['geometry']]

### Euro 24

In [ ]:
Scores_Eur_2024_752temp = Scores_Eur_2024_752[Scores_Eur_2024_752['Nuance liste'] == 'LFI'][['id_bv', '% Voix/exprimés']].assign(
    **{'% Voix/exprimés': lambda df: pd.to_numeric(df['% Voix/exprimés'].str.replace('%', '').str.replace(',', '.'), errors='coerce')}
)
BDV4_Scores_map = pd.merge(BDV_Map_75,Scores_Eur_2024_752temp,how='left',left_on=['id_bv'],right_on=['id_bv'])

In [ ]:
BDV4_Scores_map.explore("% Voix/exprimés", cmap="Reds",fill_opacity=0.15)  

In [ ]:
gdf = gpd.read_file('Source/2024/elections-europeennes-2024.geojson')

In [ ]:
gdf['abst'] = 1 -(gdf['nb_votant'] / gdf['nb_inscr'])

In [ ]:
gdf.explore('abst', cmap="Reds",fill_opacity=0.15,tooltip=['id_bv','abst'])  

### Legis 22

In [ ]:
Leg_t1_2022_75temp=Leg_t1_2022_75[Leg_t1_2022_75['Nuance']=='NUP'][['id_bv',"% Abs/Ins"]]
BDV4_Scores_map = pd.merge(BDV_Map_75,Leg_t1_2022_75temp,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [ ]:
m=BDV4_Scores_map.explore("% Abs/Ins", cmap="Blues",fill_opacity=0.15)  

In [ ]:
Leg_t1_2022_75temp=Leg_t1_2022_75[Leg_t1_2022_75['Nuance']=='NUP'][['id_bv','% Voix/Exp']]
BDV4_Scores_map = pd.merge(BDV_Map_75,Leg_t1_2022_75temp,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [ ]:
# First, ensure there are no non-numeric values in the column
BDV4_Scores_map['% Voix/Exp'] = pd.to_numeric(BDV4_Scores_map['% Voix/Exp'], errors='coerce')

# Now, convert the column to integer
BDV4_Scores_map['% Voix/Exp'] = BDV4_Scores_map['% Voix/Exp'].astype('Int64')  # Using 'Int64' to allow for NaNs

In [ ]:
BDV4_Scores_map.explore(
    m=m,
    column='% Voix/Exp', # make choropleth based on "BoroName" column
    tooltip=['% Voix/Exp',"id_bv"], # show "BoroName" value in tooltip (on hover)
    popup=True, # show all values in popup (on click)
    tiles="OpenStreetMap", # use "CartoDB positron" tiles
    cmap="Reds", # use "Set1" matplotlib colormap
    name="Score Natalie législatives 2022 (votes exprimés)",
    legend_kwds={'colorbar' : True, 'scale': True},
    show=False# name of the layer in the map
    #style_kwds=dict(color="black") # use black outline
)

folium.LayerControl(collapsed=False).add_to(m)  # use folium to add layer control

m.save('gerg.html')